In [13]:
!pip install emoji

     -------------------------------------- 358.9/358.9 kB 1.3 MB/s eta 0:00:00


In [37]:
!pip install nltk

In [54]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shahid.iqbal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import emoji
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score,fbeta_score

import warnings
warnings.filterwarnings("ignore")

In [23]:
# Load the training and test data from Excel files

train_data = pd.read_excel('G:/Desktop 1/HSE/Teaching/Fall 2023/Neural Network and Deep Learning/Lecture Slides/eng-train.xlsx')
test_data = pd.read_excel('G:/Desktop 1/HSE/Teaching/Fall 2023/Neural Network and Deep Learning/Lecture Slides/eng-test.xlsx')


In [24]:
train_data.head()

,text,label
0,and all off you die,1
1,fuck them all the world is better off without ...,1
2,death to zionist,1
3,we will exterminate these racists vermines so ...,1
4,muslims should be deported from europe before ...,1


In [25]:
 test_data.head()

,text,label
0,fuck that towelhead muslim kill obama and all ...,1
1,bring back at keyboard hitler to sort this issue,0
2,bro go kill yooyourself your probably a sad ye...,1
3,you will suffer after you dying,1
4,fuck that jew bitch europe will not survive eu...,1


In [26]:
#target classes in train file

train_data['label'].value_counts()

1    872
0     58
Name: label, dtype: int64

In [27]:
#target classes in test file

test_data['label'].value_counts()

1    243
0     16
Name: label, dtype: int64

In [28]:
# missing values in train file

train_data.isnull().sum()

text     0
label    0
dtype: int64

In [29]:
# missing values in test file

test_data.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
#Data preprocessing
#1) Removal of all the hashtags, HTML tags, mentions, punctuations, and URLs.
#2) Changing to lowercase (for case sensitive languages)
#3) Replace emojis with corresponding text

In [30]:
# Defining a pre-processing function

def preprocess_text(text):
    # Removal of hashtags, HTML tags, mentions, punctuations, and URLs
    text = re.sub(r'#\w+', '', text)             # Remove hashtags
    text = re.sub(r'<.*?>', '', text)            # Remove HTML tags
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)   # Remove mentions numbers
    text = re.sub(r'[^\w\s]', '', text)          # Remove punctuations
    text = re.sub(r'http\S+', '', text)          # Remove URLs
    
    
    # Changing to lowercase
    text = text.lower()

    # Replace emoji with corresponding text representation
    text = emoji.demojize(text)

    return text

In [32]:
# Apply Pre-processing
train_data['text']= train_data['text'].apply(lambda x: preprocess_text(x))
train_data['text'].head()

0                                  and all off you die
1    fuck them all the world is better off without ...
2                                     death to zionist
3    we will exterminate these racists vermines so ...
4    muslims should be deported from europe before ...
Name: text, dtype: object

In [33]:
test_data['text']= test_data['text'].apply(lambda x: preprocess_text(x))
test_data['text'].head()

0    fuck that towelhead muslim kill obama and all ...
1     bring back at keyboard hitler to sort this issue
2    bro go kill yooyourself your probably a sad ye...
3                      you will suffer after you dying
4    fuck that jew bitch europe will not survive eu...
Name: text, dtype: object

In [55]:
# load the stop word list
stop_words_list = set(stopwords.words('english')) 

# define the function
def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words_list]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

In [56]:
train_data['text'] = train_data['text'].apply(remove_stop_words)
train_data.head()

,text,label
0,die,1
1,fuck world better without shit country know ki...,1
2,death zionist,1
3,exterminate racists vermines easily,1
4,muslims deported europe late,1


In [57]:
test_data['text'] = test_data['text'].apply(remove_stop_words)
test_data.head()

,text,label
0,fuck towelhead muslim kill obama othe muslim f...,1
1,bring back keyboard hitler sort issue,0
2,bro go kill yooyourself probably sad year old ...,1
3,suffer dying,1
4,fuck jew bitch europe survive europe fine jewd...,1


In [ ]:
#TF-IDF weighting scheme

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(ngram_range=(1,1))
train_features = tv.fit_transform(train_data['text'])
test_features = tv.transform(test_data['text'])

In [59]:
X_train = train_features
y_train = train_data['label']
X_test  = test_features
y_test  = test_data['label']

In [60]:
X_train.shape , X_test.shape

((930, 2239), (259, 2239))

In [61]:
# uni-gram
# Set the random seed for reproducibility
random_seed = 42
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
print("Random Forest Result")
rfc = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=10,random_state=52)
pred = rfc.fit(X_train, y_train).predict(X_test)
print("RFC")
print(classification_report(y_test,pred,digits=4))

# Print the confusion matrix
cm = confusion_matrix(y_test, pred)
print("Confusion Matrix:")
print(cm)
# Extracting TP, TN, FP, and FN
tn, fp, fn, tp = cm.ravel()

print(f"True Positives (TP): {tp}")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")

Random FOrest Result
RFC
              precision    recall  f1-score   support

           0     0.6667    0.2500    0.3636        16
           1     0.9526    0.9918    0.9718       243

    accuracy                         0.9459       259
   macro avg     0.8096    0.6209    0.6677       259
weighted avg     0.9349    0.9459    0.9342       259

Confusion Matrix:
[[  4  12]
 [  2 241]]
True Positives (TP): 241
True Negatives (TN): 4
False Positives (FP): 12
False Negatives (FN): 2
